In [2]:
import pandas as pd

In [3]:
intent = pd.read_csv('intent.csv')

In [4]:
persona = pd.read_csv('persona.csv')

In [5]:
subject = pd.read_csv('subject.csv')

In [6]:
template = pd.read_csv('template.csv')

In [9]:
template = template[:5]

In [10]:
template

,unique_keyword_id,intent_localised_id,keyword_template,language
0,5137f7e3aaa9d54f24d67913786ce4ba,340cd1dcf5d3871fa2584362194cc4fe,"intent,subject",eng
1,d67a76d8468c0641ff96dd69c0844180,340cd1dcf5d3871fa2584362194cc4fe,"subject,intent",eng
2,1898cc68df3a1329ed2c7c3a1320ff94,90d991b47098ceef3d8283a386cf7e91,"intent,subject",eng
3,01d6cbc7d1212e70cd859372a7b081f0,90d991b47098ceef3d8283a386cf7e91,"subject,intent",eng
4,8ef49bdea5131365eb2ecee679824819,5a753229efb8128b16b50c38708c6638,"subject,intent",eng


In [48]:
list1 = template[:1].merge(subject, on='language')['subject_localised']

In [49]:
list1 = pd.DataFrame(list1)

In [38]:
list2 = template[:1].merge(persona, on='language')['persona_localised']

In [39]:
list2 = pd.DataFrame(list2)

In [50]:
list3 = template[:1].merge(intent, on='language')['intent_localised']

In [51]:
list3 = pd.DataFrame(list3)

In [53]:
list1['key'] = 0
list2['key'] = 0
list3['key'] = 0

In [44]:
df_temp = pd.merge(list1, list2, on='key', how='outer')

# Merge df_temp with df_persona
df_final = pd.merge(df_temp, list3, on='key', how='outer')

# Drop the key column
# df_final = df_final.drop('key', 1)

In [54]:
df_temp = pd.merge(list1, list3, on='key', how='outer')

In [55]:
df_temp.drop('key', axis=1, inplace=True)

,subject_localised,key,intent_localised
0,Arabic,0,teacher
1,Arabic,0,tutor
2,Arabic,0,speaker
3,Arabic,0,partner
4,Arabic,0,class
...,...,...,...
3707,Slovak,0,OSD
3708,Slovak,0,National
3709,Slovak,0,CILS
3710,Slovak,0,JLPT


In [ ]:
import pandas as pd
import numpy as np
from itertools import product

# Create a mapping of dimension names to dataframes
dimension_dfs = {
    'pma': pma_df,
    'subject': subject_df,
    'intent': intent_df,
    'persona': persona_df
}

def combine_dimensions(row):
    dimensions = row['keyword_template'].split(',')
    language = row['language']
    combinations = []

    for dimension in dimensions:
        if dimension in dimension_dfs:
            combinations.append(dimension_dfs[dimension].loc[dimension_dfs[dimension]['language'] == language, f'{dimension}_localised'].values.tolist())
        else:
            combinations.append([dimension])

    # Create all possible combinations
    combinations = list(product(*combinations))
    combinations = [' '.join(combination) for combination in combinations]

    return pd.Series(combinations)

# Apply combine_dimensions function to each row in keyword_template_df
keyword_template_df['combinations'] = keyword_template_df.apply(combine_dimensions, axis=1)

# Explode combinations into separate rows
keyword_template_df = keyword_template_df.explode('combinations')

# Merge keyword_template_df with campaign_template_df
merged_df = pd.merge(keyword_template_df, campaign_template_df, on='market_id')

# Function to handle campaign combinations
def campaign_combinations(row):
    campaign_template = row['campaign_template'].split(',')
    replacements = {
        'market_id': str(row['market_id']),
        'language': row['language']
        # Add more replacements as needed
    }

    # Replace elements of the campaign template as needed
    campaign_template = [replacements.get(item, item) for item in campaign_template]

    return ' '.join(campaign_template)

# Apply campaign_combinations function to each row in merged_df
merged_df['campaign_combination'] = merged_df.apply(campaign_combinations, axis=1)

# Final output dataframe
final_df = merged_df[['language', 'keyword_template', 'market_id', 'campaign_template', 'combinations', 'campaign_combination']]

    